In this exercise, we are mostly following Chapter 1-2 of "The Art of Molecular Dynamics Simulation" by D.C. Rapaport, 1995.

The goal of the exercise is to generate a minimum viable product for performing molecular dynamics simulations and to calculate properties of the ensemble using the relations of statistical mechanics. The model starts from a lattice configuration in 2D. We truncate the Lennard-Jones (6-12 potential), which represents the vdW interactions between neighboring atoms, and perform simple MD with this potential.

Robert Palmere, 2022

In [ ]:
# Import the required modules

import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

In [ ]:
eps = 10
sig = 3

def u(rij):
    """ Complete LJ 6-12 Potential
    """
    return 4*eps*((sig/rij)**12 - (sig/rij)**6)

def simple_u(rij):
    """ Truncated LJ 6-12 Potential
    """
    if rij < sig*(2**(1/6)):
        return 4*eps*((sig/rij)**12 - (sig/rij)**6) + eps
    else:
        return 0

def square_u(rij):
  """ Square-well potential
  """
      if rij < sig*(2**(1/6)) and rij >= 1: return -eps
    elif rij < 1: return 4*eps*((sig/rij)**12 - (sig/rij)**6) + eps
    else: return 0

us = [u(i) for i in np.arange(.9, 2, .01)]
u_simple = [simple_u(i) for i in np.arange(.9, 2, .01)]
u_square = [square_u(i) for i in np.arange(.9, 2, .01)]

fig, ax = plt.subplots(1, 3, figsize=(10, 3))
ax[0].plot(np.arange(.9, 2, .01), us, lw=3, c='b', label='6-12')
ax[1].plot(np.arange(.9, 2, .01), u_simple, lw=3, c='r', label='Truncated')
ax[2].plot(np.arange(.9, 2, .01), u_square, lw=3, c='tan', label='Square Well')

for a in ax:
    a.axhline(y=0, ls=':', c='k')
    a.legend()
plt.tight_layout()
plt.show()

In the following entry, define your own potential using the function template provided. Plot the result using the template plotting function. What do you expect to happen as a result of your potential?

In [3]:
# Template for returning a potential
# Change expression after the 'u =' to a potential of your choice

# Take care in generating a function that does not divide by zero at the rij 
# values accessble to the particles

def potential(rij):
    u = -np.exp(-rij**2)
    return u

In [ ]:
# Template function for plotting potentials

def plot_potential(potential, span=(0, 2)):
    """ Plot potential functions

    Note to students:
      The y-axis should adjust automatically to account for the maxima of your plot.
      If you'd like to adjust the range displayed along the x-axis, you can simply
      edit the tuple which defines the keyword argument "span" to your desired
      minimum and maximum x-axis values.

    Parameters
    ----------

    potential : lambda
      A lambda function which returns the expression for the vdW potential

    span : tuple
      The maximum and minimum values along the x-axis

    """
    fig, ax = plt.subplots()
    x = np.arange(span[0], span[1], .01)
    p = [potential(i) for i in x]
    ax.plot(x, p, lw=3)
    ax.axhline(y=0, ls=':', c='k', lw=3)
    plt.show()

plot_potential(potential) # Sigmoidal potential for example

In [ ]:
def potential(rij):
    u = 100 # < rij -dependent function here >
    return u

In [ ]:
# Generate a file containing input parameters to be passed as an argument to MD engine

with open('input', 'w') as fp:
    fp.write('runId 1\n')
    fp.write('initUcell 20 20\n')
    fp.write('density 0.8\n')
    fp.write('temperature 1\n')
    fp.write('deltaT 0.005\n')
    fp.write('randSeed 17\n')
    fp.write('stepAvg 100\n')
    fp.write('stepEquil 0\n')
    fp.write('stepLimit 25\n')

In [ ]:
class MD(object):
    """ Class to perform simple molecular dynamics
    """
    def __init__(self, fn, custom_potential=False, rcut=0.3):
        self.fn = fn
        self.custom_potential = custom_potential
        self.namelist = {'runId' : None,
                         'initUcell' : None,
                         'density' : None,
                         'temperature' : None,
                         'deltaT' : None,
                         'randSeed' : None,
                         'stepAvg' : None,
                         'stepEquil' : None,
                         'stepLimit' : None }
        
        # Simulation Parameters
        
        self.step_count = 0
        self.time_now = 0
        self.ndim = 2
        self.natom = None
        self.vmag = None
        self.region = np.empty(self.ndim)
        self.regionH = np.empty(self.ndim)
        self.rcut = rcut
        
        # Position, Velocities, Acceleration, and Energies
        
        self.r = None
        self.rv = None
        self.ra = None
        self.vsum = None
        self.vvsum = 0
        self.usum = 0
        self.virsum = 0
        self.kinenergy = 0
        self.potenergy = 0
        self.totenergy = 0
        self.pressure = 0
        
        
    def get_name_list(self):
        """ Read simulation parameters into namelist dictionary
        """
        with open(self.fn, 'r') as fp:
            lines = fp.readlines()
            for n, l in enumerate(lines):
                if l.split()[0] != 'initUcell':
                    self.namelist[l.split()[0]] = float(l.split()[1])
                else:
                    self.namelist[l.split()[0]] = [float(l.split()[1]), float(l.split()[2])]
                
        return self.namelist
    
    def set_params(self):
        """ C-style preallocation of arrays for phase space parameters
        """
        self.rcut = pow(2., 1./6.)
        for k in range(self.ndim):
            self.region[k] =  self.namelist['initUcell'][k] / np.sqrt(self.namelist['density'])
            self.regionH[k] = self.region[k] * 0.5
        self.natom = int(self.namelist['initUcell'][0] * self.namelist['initUcell'][1])
        self.vmag = np.sqrt(self.ndim * (1. - (1. / self.natom)) * self.namelist['temperature'])   
        
    def alloc_arrays(self):
        self.r = np.empty((self.ndim, int(self.natom)))
        self.rv = np.empty((self.ndim, int(self.natom)))
        self.ra = np.zeros((self.ndim, int(self.natom)))
    
    def init_coords(self):
        """ Preparation of initial state for atomic coordinates (2D grid)
        Note - This can also be done using np.meshgrid or the like
        """
        
        gap = np.empty(self.ndim)
        c = np.empty(self.ndim)
        for k in range(self.ndim):
            gap[k] = self.region[k] / self.namelist['initUcell'][k]
        
        n = -1
        for nY in range(int(self.namelist['initUcell'][1])):
            
            c[1] = (nY - 0.5) * gap[1] - self.regionH[1]
            for nX in range(int(self.namelist['initUcell'][0])):
                c[0] = (nX - 0.5) * gap[0] - self.regionH[0]
                n += 1
                for k in range(self.ndim):
                    self.r[k][n] = c[k]
    
    def init_vels(self):
        """ Preparation of initial state for atomic velocities
        """

        self.vsum = np.empty(self.ndim)
        np.random.seed(int(self.namelist['randSeed']))
        
        for k in range(self.ndim):
            if k == 0:
                self.vsum[k] = 0.
            for n in range(int(self.natom)):
                ang = 2. * np.pi * np.random.rand(1)
                self.rv[0][n] = self.vmag * np.cos(ang)
                self.rv[1][n] = self.vmag * np.sin(ang)
                for k in range(self.ndim):
                    self.vsum[k] = self.vsum[k] + self.rv[k][n]
                
        for k in range(self.ndim):
            self.vsum[k] = self.vsum[k] / self.natom
            for n in range(int(self.natom)):
                self.rv[k][n] = self.rv[k][n] - self.vsum[k]
                
    def signr(self, x, y):
        """ Reverse sign of x depending on y
        """
        if y >= 0.:
            return x
        else:
            return -x

    def compute_forces(self):
        """ Compute the forces of all interacting atom pairs < rcut 
        """
        
        rrcut = self.rcut ** 2
        dr = np.empty(self.ndim)
        rr = 0
        rri = 0
        rri3 = 0
        fcval = 0
        
        self.usum = 0
        self.virsum = 0
        for j1 in range(int(self.natom)-1):
            for j2 in range(j1+1, int(self.natom)): # Note - Computing for all atom pairs (expensive)
                
                rr = 0
                for k in range(self.ndim):
                    dr[k] = self.r[k][j1] - self.r[k][j2] # Distance between atom i and atom j along one axis
                    
                    if (abs(dr[k]) > self.regionH[k]):
                        dr[k] = dr[k] - self.signr(self.region[k], dr[k])
                    rr = rr + (dr[k]**2)

                if self.custom_potential == False:
                    if (rr < rrcut): 
                        rri = 1. / rr # As rr gets smaller, rri increases causing repulsion
                        rri3 = rri ** 3
                        fcval = 48. * rri3 * (rri3 - 0.5) * rri
                        for k in range(self.ndim):
                            f = fcval * dr[k] # setting F=0 will produce non-colliding particles
                            self.ra[k][j1] = self.ra[k][j1] + f
                            self.ra[k][j2] = self.ra[k][j2] - f
                        self.usum += 4. * rri3 * (rri3 - 1.) + 1. # Adds to potential energy
                                                                # but does nothing with it
                        self.virsum += (fcval * rr)
                else:
                    if (rr < rrcut):
                        fcval = self.custom_potential(dr[k]) # Magnitude of force
                        for k in range(self.ndim):
                            f = fcval * dr[k] 
                            self.ra[k][j1] = self.ra[k][j1] + f # Store the F=a in acceleration arrays (ignoring mass)
                            self.ra[k][j2] = self.ra[k][j2] - f

                    
    def leapfrog_step(self):
        """ Leap-frog step to the next state in phase space
        """
        for n in range(int(self.natom)):
            for k in range(self.ndim):
                self.rv[k][n] = self.rv[k][n] + self.namelist['deltaT'] * self.ra[k][n]
                self.r[k][n] = self.r[k][n] + self.namelist['deltaT'] * self.rv[k][n]
                
    def apply_boundary_cond(self):
        """ Apply boundary condition to all atoms by reversing their direction past region
        """
        
        for n in range(int(self.natom)):
            for k in range(self.ndim):
                if (self.r[k][n] >= self.regionH[k]):
                    self.r[k][n] = self.r[k][n] - self.region[k]
                elif (self.r[k][n] < -self.regionH[k]):
                    self.r[k][n] = self.r[k][n] + self.region[k]
                    
    def eval_props(self):
        """ Update the energetic properties of the system
        """
        self.vsum = 0.
        self.vvsum = 0.
        for n in range(int(self.natom)):
            vv = 0.
            for k in range(self.ndim):
                v = self.rv[k][n] - 0.5 * self.ra[k][n] * self.namelist['deltaT']
                self.vsum += v
                vv = vv + (v**2)
            self.vvsum += vv
        self.kinenergy = 0.5 * self.vvsum / self.natom
        self.potenergy = self.usum / self.natom
        self.totenergy = self.kinenergy + self.potenergy
        self.pressure = self.namelist['density'] * (self.vvsum + self.virsum) / (self.natom * self.ndim)
                      
    
    def display_props(self):
        """ Display the step count and total energy
        """
        print('Step: %d Energy: %0.2f' % (self.step_count, self.totenergy), end='\r')
    
    def single_step(self):
        """ A complete MD step 
        """
        self.step_count += 1
        self.time_now = self.step_count * self.namelist['temperature']
        self.compute_forces()
        self.leapfrog_step()
        self.apply_boundary_cond()
        self.eval_props()
        self.display_props()

In [ ]:
# Initialize engine with default (truncated LJ) potential

fig, ax = plt.subplots()
test = MD('input', custom_potential=False)
test.get_name_list()
test.set_params()
test.alloc_arrays()
test.init_coords()
test.init_vels()
test.single_step()
ax.scatter(test.r[0, :], test.r[1, :], color='k')
plt.show()

In [ ]:
from matplotlib import animation # Required class for animations
from IPython.display import HTML # Required for Jupyter Notebook display


# Animate the resulting process from the truncated LJ potential

anifig = plt.figure()
ax = plt.scatter(test.r[0, :], test.r[1, :], color='k')

def animate(i):
    test.single_step()
    ax.set_offsets(test.r.T) # update scatter data (req. transpose)
    return ax

anim = animation.FuncAnimation(anifig, animate, frames=25, interval=10)
HTML(anim.to_jshtml())

Now we try with a custom potential which we should have defined above.

In [ ]:
# Initialize engine with custom potential

fig, ax = plt.subplots()
custom = MD('input', custom_potential=potential, rcut=2.5)
custom.get_name_list()
custom.set_params()
custom.alloc_arrays()
custom.init_coords()
custom.init_vels()
custom.single_step()
ax.scatter(custom.r[0, :], custom.r[1, :], color='k')
plt.show()

In [ ]:
# Animate the resulting process from the custom potential

custom_anifig = plt.figure()
ax = plt.scatter(custom.r[0, :], custom.r[1, :], color='k')

def custom_animate(i):
    custom.single_step()
    ax.set_offsets(custom.r.T)
    return ax

anim = animation.FuncAnimation(custom_anifig, custom_animate, frames=100, interval=10)
HTML(anim.to_jshtml())